# Handling Missing Values in Python

Missing values refer to absent or undefined data entries, often represented as `NaN`, `None`, `null`, or `NA`.  
**Causes include:**
- Data corruption  
- Entry errors  
- Incomplete survey responses  

Unaddressed missing values can bias results and reduce model performance. Handling strategies include:
- **Deletion** (drop rows/columns)
- **Imputation** (fill in missing values)
- **Using tolerant algorithms** (e.g., decision trees, XGBoost)

### Common Methods for Imputing Missing Values

Imputation is the process of replacing missing values with inferred ones based on statistical or algorithmic methods. The appropriate technique depends on the data type, context, and missing data pattern (MCAR, MAR, or MNAR).

---

**1. Simple Statistical Imputation**

- **Mean/Median Imputation** (for numerical data):
  - Use **mean** if the feature is normally distributed.
  - Use **median** if the feature has outliers or is skewed.

- **Mode Imputation** (for categorical data):
  - Replace missing values with the most frequent category (mode).

---

**2. K-Nearest Neighbors (KNN) Imputation**

- Uses similarity between rows to estimate missing values.
- Works for both numerical and categorical data.
- Can capture local patterns but may be **computationally expensive**.

---

**3. Regression Imputation**

- Predicts missing values using regression models based on other features.
- Suitable when a strong linear or nonlinear relationship exists among variables.

---

**4. Tree-Based Imputation**

- Uses **Decision Trees** or **Random Forests** to model and predict missing data.
- Can:
  - Handle missing values during training.
  - Learn complex, non-linear patterns.
  - Work well with mixed data types.

---

**5. Advanced Techniques**

- **MICE (Multiple Imputation by Chained Equations)**:
  - Iteratively models each variable with missing data using the others.
  - Captures uncertainty and inter-variable relationships.

- **Deep Learning Methods**:
  - **Autoencoders** learn latent representations and reconstruct missing values.
  - Best for high-dimensional or complex datasets.

---

**6. Time-Series Specific Methods**

- **Forward-fill (`ffill`)**: Fill with the previous non-null value.
- **Backward-fill (`bfill`)**: Fill with the next non-null value.
- **Interpolation**: Estimate using linear or polynomial trends.


### Important Considerations

- **Understand the missingness pattern** before choosing a method (MCAR, MAR, MNAR).
- Imputation makes assumptions — improper techniques can **introduce bias**.
- Always validate imputation effects using **cross-validation** or **sensitivity analysis**.

### Simple Statistical Imputation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [113]:
%pwd

'/home/as/practice/Python-for-Machine-Learning'

In [114]:
%cd Python-for-Machine-Learning/

[Errno 2] No such file or directory: 'Python-for-Machine-Learning/'
/home/as/practice/Python-for-Machine-Learning


In [115]:
# Load Data
df = pd.read_csv('Datasets/airbnb_listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2992450,https://www.airbnb.com/rooms/2992450,20250609011619,2025-06-09,city scrape,Luxury 2 bedroom apartment,The apartment is located in a quiet neighborho...,NaN,https://a0.muscache.com/pictures/44627226/0e72...,4621559,...,4.56,3.22,3.67,NaN,f,1,1,0,0,0.07
1,3820211,https://www.airbnb.com/rooms/3820211,20250609011619,2025-06-09,city scrape,Restored Precinct in Center Sq. w/Parking,"Cozy, cool little 1BR Apt in the heart Albany'...","Great restaurants, architecture, walking, peop...",https://a0.muscache.com/pictures/prohost-api/H...,19648678,...,4.81,4.83,4.78,NaN,f,4,4,0,0,2.34
2,5651579,https://www.airbnb.com/rooms/5651579,20250609011619,2025-06-09,city scrape,Large studio apt by Capital Center & ESP@,"Spacious studio with hardwood floors, fully eq...",The neighborhood is very eclectic. We have a v...,https://a0.muscache.com/pictures/b3fc42f3-6e5e...,29288920,...,4.88,4.76,4.64,NaN,f,2,1,1,0,3.02
3,6623339,https://www.airbnb.com/rooms/6623339,20250609011619,2025-06-09,city scrape,Center Sq. Loft in Converted Precinct w/ Parking,Large renovated 1 bedroom apartment in convert...,"Located in Albany's finest urban neighborhood,...",https://a0.muscache.com/pictures/prohost-api/H...,19648678,...,4.70,4.80,4.72,NaN,f,4,4,0,0,2.72
4,9005989,https://www.airbnb.com/rooms/9005989,20250609011619,2025-06-09,city scrape,"Studio in The heart of Center SQ, in Albany NY",(21 years of age or older ONLY) NON- SMOKING.....,"There are many shops, restaurants, bars, museu...",https://a0.muscache.com/pictures/d242a77e-437c...,17766924,...,4.93,4.87,4.77,NaN,f,1,1,0,0,5.76


In [116]:
df.isnull().sum()

id                                               0
listing_url                                      0
scrape_id                                        0
last_scraped                                     0
source                                           0
                                                ..
calculated_host_listings_count                   0
calculated_host_listings_count_entire_homes      0
calculated_host_listings_count_private_rooms     0
calculated_host_listings_count_shared_rooms      0
reviews_per_month                               67
Length: 79, dtype: int64

In [117]:
## display columns only with missing values
df.isnull().sum()[df.isnull().sum() > 0]

description                      10
neighborhood_overview           249
host_location                   108
host_about                      206
host_response_time               26
host_response_rate               26
host_acceptance_rate              8
host_is_superhost                12
host_neighbourhood               71
neighbourhood                   249
neighbourhood_group_cleansed    447
bathrooms                        32
bedrooms                          5
beds                             32
price                            32
calendar_updated                447
has_availability                  6
estimated_revenue_l365d          32
first_review                     67
last_review                      67
review_scores_rating             67
review_scores_accuracy           67
review_scores_cleanliness        67
review_scores_checkin            67
review_scores_communication      67
review_scores_location           67
review_scores_value              67
license                     

In [118]:
# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = df.select_dtypes(include=["object", "category"]).columns

numeric_cols


Index(['id', 'scrape_id', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'availability_eoy', 'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'license',
       'calculated_host_listings_count',
       'calculate

In [119]:
categorical_cols

Index(['listing_url', 'last_scraped', 'source', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type',
       'bathrooms_text', 'amenities', 'price', 'has_availability',
       'calendar_last_scraped', 'first_review', 'last_review',
       'instant_bookable'],
      dtype='object')

In [120]:
#  Impute numeric features
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        if df[col].skew() < 0.5: 
            imputed_value = df[col].mean()
            method_used = "mean"
        else:
            imputed_value = df[col].median()
            method_used = "median"
        df[col].fillna(imputed_value, inplace=True)
        print(f"[✔] {col}: Missing values imputed using {method_used} ({imputed_value})")

[✔] neighbourhood_group_cleansed: Missing values imputed using median (nan)
[✔] bathrooms: Missing values imputed using median (1.0)
[✔] bedrooms: Missing values imputed using median (1.0)
[✔] beds: Missing values imputed using median (1.0)
[✔] calendar_updated: Missing values imputed using median (nan)
[✔] estimated_revenue_l365d: Missing values imputed using median (5280.0)
[✔] review_scores_rating: Missing values imputed using mean (4.745078947368421)
[✔] review_scores_accuracy: Missing values imputed using mean (4.788684210526316)
[✔] review_scores_cleanliness: Missing values imputed using mean (4.772342105263158)
[✔] review_scores_checkin: Missing values imputed using mean (4.825868421052632)
[✔] review_scores_communication: Missing values imputed using mean (4.848236842105263)
[✔] review_scores_location: Missing values imputed using mean (4.664342105263159)
[✔] review_scores_value: Missing values imputed using mean (4.709789473684211)
[✔] license: Missing values imputed using med

/tmp/ipykernel_1633/202683654.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(imputed_value, inplace=True)


In [122]:
# Impute categorical features
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0]
        df[col].fillna(mode_value,inplace=True) 
        print(f"[✔] {col}: Missing values imputed using mode ('{mode_value}')")

[✔] description: Missing values imputed using mode ('Charming 5 room bed and breakfast located across from Washington Park in Albany, NY. Each room has a private bathroom and workstation. Guests have access to a shared kitchen, dining area, living space and small office nook with printer.')
[✔] neighborhood_overview: Missing values imputed using mode ('The Argus Hotel is located directly on Washington Park (Olmstead Design) is just a short walk to downtown, the Capitol Building and Empire State Plaza, Albany Medical Center, Albany Law School & Albany Pharmacy College, so leave your car for the day in our onsite parking lot. The hotel is only two blocks from hip Center Square, a neighborhood of 19th century brownstones, where you can stroll to numerous restaurants, galleries, pubs, and music venues.')
[✔] host_location: Missing values imputed using mode ('Albany, NY')
[✔] host_about: Missing values imputed using mode ('Hi my name is Scott Guadagnino. I’ve been in the Albany area for 9 y

/tmp/ipykernel_1633/2484694464.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value,inplace=True)


In [123]:
# drop columns with all missing values
df.drop(columns=["calendar_updated", "license", "neighbourhood_group_cleansed"], inplace=True)

In [124]:
# Final Check
print("\n✅ Remaining missing values:\n", df.isnull().sum()[df.isnull().sum() > 0])


✅ Remaining missing values:
 Series([], dtype: int64)


### K-Nearest Neighbors (KNN) Imputation

In [22]:
from sklearn.impute import KNNImputer

In [23]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [24]:
df.isnull().sum().sort_values(ascending=False)

deck           688
age            177
embarked         2
embark_town      2
sex              0
pclass           0
survived         0
fare             0
parch            0
sibsp            0
class            0
adult_male       0
who              0
alive            0
alone            0
dtype: int64

In [25]:
imputer = KNNImputer(n_neighbors=4, weights="uniform") # You can tune n_neighbors
df['age'] = imputer.fit_transform(df[['age']]) # Impute age column 

In [26]:
df.isnull().sum().sort_values(ascending=False)

deck           688
embarked         2
embark_town      2
age              0
survived         0
pclass           0
sex              0
fare             0
parch            0
sibsp            0
class            0
adult_male       0
who              0
alive            0
alone            0
dtype: int64